In [2]:
import pandas as pd


In [16]:
test_df = pd.read_parquet("results_last-rpiembeddor_esm_rnafm, lr: 0.001, wd: 0.1, dr: 0.3, seed: 999_rpi2825_test_set.parquet", "pyarrow")

In [17]:
test_df.isnull().any()  

predictions    False
labels         False
dtype: bool

In [5]:
df = pd.read_csv('xrpi_f_with_logits.csv')
df.head()
df.columns

Index(['Unnamed: 0', 'label', 'predictions', 'scores'], dtype='object')

In [6]:
import pandas as pd

# Path to the CSV file
csv_file_path = 'xrpi_f_with_logits.csv'
# Path for the output Parquet file
parquet_file_path = 'xrpi_f.parquet'

df = pd.read_csv('xrpi_f_with_logits.csv') \
    .drop('Unnamed: 0', axis=1) \
    .drop('predictions', axis=1) \
    .rename(columns={'label': 'labels', 'scores': 'logits'}) \
    .astype({'labels': int, 'logits': float})

df.to_parquet('xrpi_f.parquet', index=False)


In [8]:
a = pd.read_parquet('xrpi_f.parquet', engine='pyarrow')

In [9]:
a.head()

,labels,logits
0,1,0.9112
1,1,0.9300
2,1,0.8287
3,1,0.9401
4,1,0.8959


In [38]:
df['predictions'].value_counts()

predictions
1    11832
0     1171
Name: count, dtype: int64

In [21]:
u = test_df["predictions"].nunique()

In [23]:
preds = test_df['predictions'].value_counts()
print(preds)

predictions
1    814
0     57
Name: count, dtype: int64


In [1]:
import pandas as pd

In [5]:
df = pd.read_parquet('data/auroc/rpiembeddor_esm_rna_fm_f_seed: 17.parquet', engine='pyarrow')
df.head()

,logits,labels
0,0.999988,1
1,0.903186,0
2,0.000165,0
3,0.000003,1
4,0.351452,0


In [11]:
import numpy as np
threshold = 0.5
results = [pd.read_parquet('data/auroc/rpiembeddor_esm_rna_fm_f_seed: 17.parquet', engine='pyarrow'), 
           pd.read_parquet('data/auroc/rpiembeddor_esm_rna_fm_f_seed: 999.parquet', engine='pyarrow'),
           pd.read_parquet('data/auroc/rpiembeddor_esm_rna_fm_f_seed: 6844.parquet', engine='pyarrow')]
dfs = []

for df in dfs:
    df['predicted_labels'] = (df['logits'] >= threshold).astype(int)
    true_positives = ((df['predicted_labels'] == 1) & (df['labels'] == 1)).sum()
    true_negatives = ((df['predicted_labels'] == 0) & (df['labels'] == 0)).sum()
    total_positives = (df['labels'] == 1).sum()
    total_negatives = (df['labels'] == 0).sum()
    results.append((true_positives, total_positives, true_negatives, total_negatives))

# Convert to NumPy array for easier averaging
results_np = np.array(results)

# Calculate averages
avg_results = results_np.mean(axis=0)

print(f"Average correctly predicted positive interactions: {avg_results[0]:.2f} out of {avg_results[1]:.2f}")
print(f"Average correctly predicted negative interactions: {avg_results[2]:.2f} out of {avg_results[3]:.2f}")

TypeError: unsupported format string passed to numpy.ndarray.__format__